Použil jsem dataset GTSRB - German Traffic Sign Recognition Benchmark a část kódu (pro zpracování vstupu) z notebooku German sign detection DNN solution https://www.kaggle.com/whitelord/german-sign-detection-dnn-solution

In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os
from keras.utils import to_categorical

In [ ]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 30
width = 30
channels = 3
classes = 43
n_inputs = height * width*channels

for i in range(classes) :
    path = "../input/train/{0}/".format(i)
    #print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print(" ")
            
Cells=np.array(data)
labels=np.array(labels)

s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [ ]:

(X_train,X_val)=(Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))])
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=(labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))])


y_trainTemp = y_train

y_train = to_categorical(y_train, classes)
y_val = to_categorical(y_val, classes)


    


Příklady některých obrázků - náhodně vybraných 25 obrázků

In [ ]:
def show_example(train_x, train_y,class_names):
    plt.figure(figsize=(10,10))
    for i in range(25):
        x = int(np.random.random()*len(train_x))
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        rgb = cv2.cvtColor(train_x[x], cv2.COLOR_BGR2RGB)
        plt.imshow(rgb, cmap=plt.cm.binary)
        plt.xlabel(class_names[train_y[x]])
    plt.show()

In [ ]:
show_example(X_train, y_trainTemp, range(43))

In [ ]:


from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:

epochs = 5
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))



In [ ]:
plt.figure(0)
plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
y_test=pd.read_csv("../input/Test.csv")
labels=y_test['Path'].as_matrix()
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image=cv2.imread('../input/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
pred = model.predict_classes(X_test)

In [ ]:
#Accuracy with the test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

# Vizualization of the confusion matrix.

In [ ]:
from sklearn.preprocessing import normalize

predictions = model.predict(X_test)
conf_matrix = np.zeros((classes, classes))
for idx, pred in enumerate(predictions):
    row = y_test[idx]
    col = np.argmax(pred)
    conf_matrix[row, col] += 1

# print(conf_matrix)
conf_matrix = normalize(conf_matrix, axis=1, norm='l1')
# print(conf_matrix)

plt.figure(figsize=(classes,classes))

plt.imshow(conf_matrix)

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        text = plt.text(j, i, "{:.2f}".format(conf_matrix[i, j]), ha="center", va="center", color="w")

plt.xticks(range(classes), range(classes))
plt.yticks(range(classes), range(classes))

plt.tight_layout()
plt.show()     

Pokusil jsem se vytvořit použítelný model s použitím co nejmenšího množství vrstev. Zajímalo mě, jak dobrá bude přesnost. Výsledky mě opravdu překvapily. Během trénování model dosáhl 98% přesnosti na trénovací sadě a 96,7% přesnosti na validační sadě. Na testovací sadě model dosáhl 89% přesnosti (pořád lepší než někteří řidiči). V confusion matici můžeme vidět shluky u prvních 8 kategorií a uprostřed kolem 20. - 30. kategorie. První skupina jsou značky maximální povolené rychlosti - liší se pouze číslem. Druhá skupina jsou Výstražné dopravní značky - červené trojúhelníky - ty se liší pouze symbolem/obrázkem uprostřed. Obě skupiny obsahují velmi podobné značky, dává tedy smysl, že se model spletl nejčastěji u těchto kategorií.

